# Исследование надежности заемщиков.

От банка представлены данные со статистикой платежеспособности клиентов.
Необходимо проанализировать как те или иные характеристики клиента банка влияют на возврат кредита в срок, а именно:
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Задание будем выполнять в несколько этапов:
1. [Обзор данных](#start)
2. [Предобработка данных:](#fiilna)
    - [заполнение пропусков](#fillna)
    - [проверка данных на аномалии и их исправление](#median)
    - [изменение типов данных](#int)
    - [удаление дупликатов](#duplicate)
    - [формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#df)
    - [категоризация дохода](#def_inccome)
    - [катекоризация целей кредита](#def_purpose)
3. [Анализ данных, ответы на поставленные вопросы](#agg)
4. [Общий вывод](#answer)

**Наименование колонок:**
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

 # Шаг 1. Обзор данных 

In [177]:
import pandas as pd
import seaborn as sb

In [178]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [179]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Наименование столбцов написано корректно

In [180]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


Таблица содержит 21525 строк и 12 столбцов, название колонок написано верно, при этом содержит пропуски в столбцах `days_employed` и `total_income`.

Сразу можем заметить, что столбец трудого стажа содержит отрицательные значения, которые необходимо исправить. Стандартное отклонение в столбце `days_employed` крайне большое и составляет 385 лет (140827 дней), что странно.

## Шаг 2.1 Заполнение пропусков  <a id="fillna"></a>

In [181]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` есть пропуски

In [182]:
'{:.1%}'.format(data['days_employed'].isna().sum()/data['children'].count())

'10.1%'

In [183]:
'{:.1%}'.format(data['total_income'].isna().sum()/data['children'].count())

'10.1%'

- Пропуски в обоих столбцах составляют 10,1% от общего количества значений
- Пропуски могут быть, как технологическими (возникшими при копировании, считывании, смене формата), так и связанные с человеческим фактором
- Заполнить пропуски следует медианными значениями, так как среднее значение для таких показателей, как трудовой стаж и ежемесячный доход не отражают действительных данных, ввиду большого разброса значений

In [184]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Пропуски в столбце `total_income` заполненны медианным значением

## Шаг 2.2 Проверка данных на аномалии и исправления. <a id="median"></a>

In [185]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В колонке встречаются значения с верхним регистром и повторы, приведем все значения к одной форме

In [186]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [187]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [188]:
data[data['children'] == -1]['children'].count()

47

В столбце `children` 47 раз встречается отрицательное значение (-1). Такого быть не может, видимо, опечатка. Стоит заменить на занчение по модулю.

In [189]:
data['children'] = abs(data['children'])
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [190]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [191]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [192]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

In [193]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [194]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [195]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Есть неопределенное значение пола `XNA` 

In [196]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


К категории пола `XNA` относиться одна строка, по представленным данным определить пол не представляется возможным. Оставим без изменения.

In [197]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [198]:
data['debt'].unique()

array([0, 1])

In [199]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Цель `ремонт жилью` стоит переименовать 

In [200]:
data['purpose'] = data['purpose'].replace('ремонт жилью', 'ремонт жилья')

Отрицательные значения могут находиться не только в столбце `days_employed`, но и в столбце `total_income`, поэтому стоит заменить данные значения по модулю

In [201]:
data['days_employed'] = abs(data['days_employed'])
data['total_income'] = abs(data['total_income'])
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Аномальные значения - значения, не соответсвующие действительности. Непроверенными остались 2 столбца, в которых могут находиться аномалии

Например, в представленных данных столбец `days_employed` отражает общий трудовой стаж в днях. Соотвественно, трудовой стаж не может быть больше возраста клиента, что стоит проверить. 

Также необходимо проверить столбец `total_income` на наличие аномальных значений (к примеру, сумма дохода более 10 млн. в месяц).

In [202]:
data[(data['days_employed'])>((data['dob_years'])*365)]['days_employed'].count()

3519

У 3529 клиентов трудовой стаж больше их возраста, что не может быть в действительности. Если в дальнейшем анализе данный столбец будет необходим, эти показатели стоит заменить.

In [203]:
data[data['total_income']>3000000]['total_income'].count()

0

In [204]:
data[data['total_income']>2000000]['total_income'].count()

2

У двух клиентов ежемесячный доход от 2 до 3 млн., что вполне может соответсвовать действительности, поэтому аномальных значений в данном столбце нет.

**Причины появления аномалий такие же, как и с пропусками, вызванные технологическим или человеческим факторами**

Заполним пропуски в столбце `days_employed` медианными значениями

In [205]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Шаг 2.3. Изменение типов данных. <a id="int"></a>


In [206]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 13.5 MB


In [207]:
data['total_income'] = data['total_income'].astype('int32')
data['children'] = data['children'].astype('int8')
data['days_employed'] = data['days_employed'].astype('float32')
data['dob_years'] = data['dob_years'].astype('int8')
data['education_id'] = data['education_id'].astype('int8')
data['family_status_id'] = data['family_status_id'].astype('int8')
data['debt'] = data['debt'].astype('int8')
data['education'] = data['education'].astype('category')
data['family_status'] = data['family_status'].astype('category')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')

data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int8    
 1   days_employed     21525 non-null  float32 
 2   dob_years         21525 non-null  int8    
 3   education         21525 non-null  category
 4   education_id      21525 non-null  int8    
 5   family_status     21525 non-null  category
 6   family_status_id  21525 non-null  int8    
 7   gender            21525 non-null  category
 8   income_type       21525 non-null  category
 9   debt              21525 non-null  int8    
 10  total_income      21525 non-null  int32   
 11  purpose           21525 non-null  object  
dtypes: category(4), float32(1), int32(1), int8(5), object(1)
memory usage: 3.8 MB


Изменение типов данных позволлило сэкономить память на 71.85% (было 13.5 Мб, стало 3.8 Мб).

Так, мы изменили данные типа `int64` на данные типа `int32`, `int8` в соотвествии с их диапозоном данных.

Также изменили данные `object` на `category`, в столбцах, содержащих категориальные значения.



## Шаг 2.4. Удаление дубликатов. <a id="duplicate"></a>

In [208]:
data.duplicated().sum()

71

В таблице присутсвует 71 дубликат строк, удалим их 

In [209]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

В таблице находились явные и неявные дубликаты. Неявные дубликаты находили с помщью метода `unique()` (например, в столбце `education`, где присутсвовали одни и теже названия, записанные с верхним и нижним регистром), которые обработали путем приведения всех наименований к нижнему регистру. Явные дубликаты исправили методом `drop_duplicates()`, удалив повторы строк.

Возможные причины появления дубликатов:
- ошибки в записи (использовались разные регистры, как в столбце `education`) 
- возможны и технические причины ошибок (неудачно написанный код мог испортить)

## Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма. <a id="df"></a>

In [210]:
education = data[['education', 'education_id']]
education.drop_duplicates().reset_index(drop=True)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [211]:
family = data[['family_status', 'family_status_id']]
family.drop_duplicates().reset_index(drop=True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [212]:
data = data.drop(['family_status', 'education'], axis=1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.672852,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803711,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422852,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747070,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.062500,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Создали отдельные Датафреймы, удалили дупликаты. Из исходной таблицы удалили столбцы `family_status` и `education`. 

### Шаг 2.6. Категоризация дохода. <a id="def_income"></a>

In [213]:
def income_category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(income_category)

Добавили столбец `total_income_category`, в котором хранятся значения по категории доходов

## Шаг 2.7. Категоризация целей кредита. <a id="def_purpose"></a>

In [214]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [215]:
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил'in purpose:
        return 'операции с недвижимостью'
    elif 'свадь' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

Добавили столбец `purpose_category`, в котором хранятся значения по целям кредита

## Ответы на вопросы. <a id="agg"></a>

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [216]:
data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})

debt                  
          count     sum      mean
children                         
0         14091  1063.0  0.075438
1          4855   445.0  0.091658
2          2052   194.0  0.094542
3           330    27.0  0.081818
4            41     4.0  0.097561
5             9     0.0  0.000000
20           76     8.0  0.105263

In [217]:
data[data['children'] != 0]['debt'].mean()

0.09208203178052424

##### Вывод 1:

Столбец `mean` отражает процент клиетов, имеющих задолжности по возврату кредита(посокльку 1 - есть задолжность, 0 - нет задолженности, а среднее отражает отношение тех, у кого есть задолженность ко всему количеству клиентов данной категории). 

**Как видим, количество детей не влияет на возврат кредита:** 
- 7,5% имеют задолжность от всего количества клиентов, не имеющих детей
- 9% имеют задолжность от всего количества клиентов, у которых 1 ребенок
- 9,5% имеют задолжность от всего количества клиентов, у которых 2 ребенока
- 8,2% имеют задолжность от всего количества клиентов, у которых 3 ребенока
- 9,8% имеют задолжность от всего количества клиентов, у которых 4 ребенока
- 10,5% имеют задолжность от всего количества клиентов, у которых 20 детей

Удивитильно, что клиенты, у которых 5 детей, не имеют задолжностей. Однако таких клиентов всего 9, что крайне мало по сравнению со всей выборкой, поэтому выводы по данной категории не является репрезентативными 

При этом, зависимости между наличием детей (неважно сколько) и возвратом кредита в срок также нет, так как процент клиентов, у которых есть дети и задолжность составляет 9,2%, против 7,5%, у которых детей нет.


##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [218]:
data.groupby('family_status_id').agg({'debt': ['count', 'sum', 'mean']})

debt                 
                  count    sum      mean
family_status_id                        
0                 12339  931.0  0.075452
1                  4151  388.0  0.093471
2                   959   63.0  0.065693
3                  1195   85.0  0.071130
4                  2810  274.0  0.097509

##### Вывод 2:

Столбец `mean` отражает процент клиетов, имеющих задолжности по возврату кредита(посокльку 1 - есть задолжность, 0 - нет задолженности, а среднее отражает отношение тех, у кого есть задолженность ко всему количеству клиентов данной категории). 

**Как видим, семейное положение не влияет на возврат кредита:** 
- 7,5% имеют задолжность от всего количества клиентов, которые женаты/замужем
- 9,3% имеют задолжность от всего количества клиентов, которые состоят в гражданском браке
- 6,5% имеют задолжность от всего количества клиентов, являющихся вдовцом/вдовой
- 7,1% имеют задолжность от всего количества клиентов, которые находятся в разводе
- 9,8% имеют задолжность от всего количества клиентов, которые не замужем/не женаты


Максимальная разница показателей составляет 3,3%, что является несущественным. 


##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [219]:
data.groupby('total_income_category').agg({'debt': ['count', 'sum', 'mean']})

debt                  
                       count     sum      mean
total_income_category                         
A                         25     2.0  0.080000
B                       5041   356.0  0.070621
C                      16016  1360.0  0.084915
D                        350    21.0  0.060000
E                         22     2.0  0.090909

##### Вывод 3:

Столбец `mean` отражает процент клиетов, имеющих задолжности по возврату кредита(посокльку 1 - есть задолжность, 0 - нет задолженности, а среднее отражает отношение тех, у кого есть задолженность ко всему количеству клиентов данной категории). 

**Уровень дохода не влияет на возврат кредита:** 
- 8% имеют задолжность от всего количества клиентов с доходом до 30 тыс.
- 7,1% имеют задолжность от всего количества клиентов с доходом от 30001 до 50000
- 8,5% имеют задолжность от всего количества клиентов с доходом от 50001 до 200000
- 6% имеют задолжность от всего количества клиентов с доходом от 200001 до 1 млн.
- 9,1% имеют задолжность от всего количества клиентов с доходом свыше 1 млн.


Максимальная разница показателей составляет 3,1%, что также является несущественным. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [220]:
data.groupby('purpose_category').agg({'debt': ['count', 'sum', 'mean']})

debt                 
                          count    sum      mean
purpose_category                                
операции с автомобилем     4306  403.0  0.093590
операции с недвижимостью  10811  782.0  0.072334
получение образования      4013  370.0  0.092200
проведение свадьбы         2324  186.0  0.080034

##### Вывод 4:

Столбец `mean` отражает процент клиетов, имеющих задолжности по возврату кредита(посокльку 1 - есть задолжность, 0 - нет задолженности, а среднее отражает отношение тех, у кого есть задолженность ко всему количеству клиентов данной категории). 

**Цель кредита не влияет на его возврат:** 
- 9,3% имеют задолжность от всего количества клиентов с целью "операции с автомобилем"
- 7,2% имеют задолжность от всего количества клиентов с целью "операции с недвижимостью"
- 9,2% имеют задолжность от всего количества клиентов с целью "проведение свадьбы"
- 8% имеют задолжность от всего количества клиентов с целью "получение образования"


Максимальная разница показателей составляет 2,3%, что также является несущественным. 

## Общий вывод: <a id="answer"></a>

Зависимости между количеством детей, семейным положением, уровнем дохода, а также целью кредита и возвратом кредита в срок нет.
Разница между всеми группами не критическая и составляет несколько процентов.